<a href="https://colab.research.google.com/github/Michila0/pose-detection/blob/main/VedioConvertImages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Upload the Video to Google Colab

In [ ]:
from google.colab import files
uploaded = files.upload()

# Install Required Libraries

In [ ]:
!pip install opencv-python

# Extract Frames from the Video

In [ ]:
import cv2
import os
from google.colab.patches import cv2_imshow

# Load the video
video_path = list(uploaded.keys())[0]  # Get the uploaded video file name
cap = cv2.VideoCapture(video_path)

# Create a folder to save extracted images
image_save_folder = "extracted_images"
if not os.path.exists(image_save_folder):
    os.makedirs(image_save_folder)

# Extract frames
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Save every nth frame
    if frame_count % 1 == 0:
        image_path = os.path.join(image_save_folder, f"frame_{frame_count}.jpg")
        cv2.imwrite(image_path, frame)
        print(f"Saved {image_path}")

    frame_count += 1

cap.release()
print(f"Extracted {frame_count} frames.")

#  View Extracted Images

In [ ]:
import matplotlib.pyplot as plt
import os

# List all extracted images
image_files = os.listdir(image_save_folder)

# Display the first few images
for i, image_file in enumerate(image_files[:5]):  # Display first 5 images
    image_path = os.path.join(image_save_folder, image_file)
    image = cv2.imread(image_path)
    plt.figure(figsize=(10, 5))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(image_file)
    plt.axis('off')
    plt.show()

# Pose Estimation + Motion Analysis
This method uses pose estimation to track the batsman's body joints and analyzes their motion to detect batting actions.

## Install Libaries

In [ ]:
!pip install mediapipe opencv-python numpy matplotlib

## Detect Batting Pose with MediaPipe
Use MediaPipe to track the batsman's body joints(e.g: arms, shoulders, wrists) and analyze their movement over time.


In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt

#Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

# Process frames from the extracted images folder
image_save_folder = "extracted_images"
image_files = sorted(os.listdir(image_save_folder))

# Store joint angles or positions
batting_frames = []

for image_file in image_files:
    image_path = os.path.join(image_save_folder, image_file)
    frame = cv2.imread(image_path)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect pose
    results = pose.process(frame_rgb)

    if results.pose_landmarks:
        # Extract key joints (e.g., right wrist, left wrist, shoulders)
        landmarks = results.pose_landmarks.landmark
        right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
        left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

        # Calculate bat swing angle (example logic)
        angle = np.arctan2(right_wrist[1] - left_wrist[1], right_wrist[0] - left_wrist[0])

        # Detect swing (threshold-based)
        if abs(angle) > 1.5:  # Adjust based on your video
            batting_frames.append(image_file)
            print(f"Batting action detected in {image_file}")

pose.close()

## Visualize Batting Action
Plot frames where batting is detected:

In [ ]:
for batting_frame in batting_frames[:3]:  # Show first 3 detected frames
    image_path = os.path.join(image_save_folder, batting_frame)
    image = cv2.imread(image_path)
    plt.figure(figsize=(10, 5))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title("Batting Action")
    plt.axis('off')
    plt.show()

# Deep Learning Action Recognition
Use a pre-trained action recognition model to classify batting actions in short video clips.

## Install MMAction2 (Action Recognition Library)

In [ ]:
# Install MMAction2 and dependencies
!pip install mmcv-full
!pip install mmaction2

## Load a Pre-Trained Model
Use a model like **SlowFast** or **I3D** for action recognition:

In [ ]:
from mmaction.apis import inference_recognizer, init_recognizer

# Load the SlowFast model (pre-trained on Kinetics-400)
config_file = 'configs/recognition/slowfast/slowfast_r50_4x16x1_256e_kinetics400_rgb.py'
checkpoint_file = 'https://download.openmmlab.com/mmaction/recognition/slowfast/slowfast_r50_4x16x1_256e_kinetics400_rgb/slowfast_r50_4x16x1_256e_kinetics400_rgb_20200704-bcde7ed7.pth'
model = init_recognizer(config_file, checkpoint_file, device='cuda:0')

## Classify Batting Action
Convert your frames into a video clip and run inference:

In [ ]:
# Example: Predict action for a clip
video_path = '/content/Copy of Cover Drive00000000 (2).mp4'  # Use your video
results = inference_recognizer(model, video_path)

# Print top-5 predicted actions
print(results)

# Object Detection
Use YOLO to detect the bat and player, then track the bat’s motion to identify swings.

## Install YOLOv5

In [ ]:
!git clone https://github.com/ultralytics/yolov5
!pip install -r yolov5/requirements.txt

## Detect Bat and Player

In [ ]:
import torch

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Detect objects in frames
bat_frames = []
for image_file in image_files:
    image_path = os.path.join(image_save_folder, image_file)
    results = model(image_path)

    # Check if "bat" or "sports ball" is detected
    if 'sports ball' in results.pandas().xyxy[0]['name'].values:
        bat_frames.append(image_file)
        print(f"Bat detected in {image_file}")